In [8]:
from pathlib import Path
from tqdm.notebook import tqdm
import pandas as pd
import json
import syntok.segmenter as segmenter
import re

In [14]:
path = Path('../../Lab1/blbook_explorer/json/')
save_to = Path('bl_books_data_ext')
save_to.mkdir(exist_ok=True)

In [15]:
#pattern = re.compile(r'(?:\bmachine[s]{,1}\b|\bengine[s]{,1}\b)',re.I)
#pattern = re.compile(r'(?:\bwom[ea]n\b|\bboy[s]{,1}\b|\bart[ie][zs]an\b|\bslave[s]{,1}\b)')
#pattern = re.compile(r'(?:\bgirl[s]{,1}\b)')
pattern = re.compile(r'(?:\bgirl[s]{,1}\b|\bart[ie][zs]an[s]{,1}\b)')
#pattern.findall('machine Machines Mdachine dmachines')

In [16]:
json_files = list(path.glob('**/*.json'))
len(json_files)

63985

In [17]:
def harvest_sentences(text,path, regex_pattern):
    if regex_pattern.findall(text):
        sentences = [' '.join([t.value for t in s]) 
                            for p in segmenter.process(text) for s in p
                                ]
        sentences = ["<START>"] + sentences + ["<END>"]
        
        target_sents = [(path,
                        sentences[i-1],
                        sentences[i],
                        sentences[i+1],
                        '<SEP>'.join(
                                set(regex_pattern.findall(s)))
                                     )
                                    for i,s in enumerate(sentences) 
                                             if regex_pattern.findall(s)
                                ]
        return target_sents
    return None

In [18]:
count = 0

rows = []

for js in tqdm(json_files):
    with open(js, 'r') as in_json:
        data = json.load(in_json)
        pages = [str(t) for p,t in data]
        if pattern.findall(' '.join(pages)):
            pages = ["<START>"] + pages + ['<END>']
            for j,page in enumerate(pages):
                
                if pattern.findall(page):
                    
                    text = ' '.join([pages[j-1],pages[j],pages[j+1]])
                    target_sentences = harvest_sentences(text,js.name,regex_pattern=pattern)
                    if target_sentences:
                        
                        rows.extend(target_sentences)
                        count+=len(target_sentences)
                
df = pd.DataFrame(rows,columns=['article_path','previous_sentence','target_sentence','next_sentence','hits'])
df.to_csv(save_to / 'bl_books_extension_words.csv')
                    
print(count)


1799907
